# AG2: Orchestrating agents (Ending a chat)

- Orchestrating agents<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/orchestration/orchestrations
- Ending a chat<br>
  - https://docs.ag2.ai/docs/user-guide/basic-concepts/orchestration/ending-a-chat

## Orchestrating agents

- **Two-agent chat**: The simplest form of conversation pattern where two agents chat back-and-forth with each other. This has been demonstrated in the previous examples.
- **Sequential chat**: A sequence of chats, each between two agents, chained together by a carryover mechanism (which brings the summary of the previous chat to the context of the next chat). Useful for simple sequential workflows.
- **Group chat**: A chat with more than two agents with options on how agents are selected. See GroupChat Overview for further details.
- **Nested chat**: A mechanism to package a workflow into a single agent/chat for reuse in a workflow.
- **Swarm**: A pattern based on agents with handoffs. There’s a shared context and each agent has tools and the ability to transfer control to other agents. The original swarm concept was created by OpenAI.

## SETUP

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables (for API key)
load_dotenv()

# Set up OpenAI API key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable or add it to a .env file")

# Define the model to use
MODEL_GPT = "gpt-4o-mini"

## Ending a chat
There are a number of ways a chat can end:
1. The maximum number of turns in a chat is reached
2. An agent’s termination function passes on a received message
3. An agent automatically replies a maximum number of times
4. A human replies with ‘exit’ when prompted
5. In a group chat, there’s no next agent
6. In a swarm, transitioning to AfterWorkOption.TERMINATE
7. Custom reply functions

### 1. Maximum turns

```python
# GroupChat with a maximum of 5 rounds
groupchat = GroupChat(
    agents=[agent_a, agent_b, agent_c],
    speaker_selection_method="round_robin",
    max_round=5,
    ...
)
gcm = GroupChatManager(
    ...
)
agent_a.initiate_chat(gcm, "first message")
# 1. agent_a with "first message" > 2. agent_b > 3. agent_c > 4. agent_a > 5. agent_b > end

# Swarm with a maximum of 5 rounds
initiate_swarm_chat(
    agents=[agent_a, agent_b, agent_c],
    max_round=5,
    messages="first message"
    ...
)
# When initial agent is set to agent_a and agents hand off to the next agent.
# 1. User with "first message" > 2. agent_a > 3. agent_b > 4. agent_c > 5. agent_a > end
```

```python
# initiate_chat with a maximum of 2 turns across the 2 agents (effectively 4 steps)
agent_a.initiate_chat(
    recipient=agent_b,
    max_turns=2,
    message="first message"
)
# 1. agent_a with "first message" > 1. agent_b > 2. agent_a > 2. agent_b > end
```

### 2. Terminating message

```python
agent_a = ConversableAgent(
    system_message="You're a helpful AI assistant, end your responses with 'DONE!'"
    ...
)

# Terminates when the agent receives a message with "DONE!" in it.
agent_b = ConversableAgent(
    is_termination_msg=lambda x: "DONE!" in (x.get("content", "") or "").upper()
    ...
)

# agent_b > agent_a replies with message "... DONE!" > agent_b ends before replying
```

### 3. Number of automatic replies

```python
agent_a = ConversableAgent(
    max_consecutive_auto_reply=2
    ...
)

agent_b = ConversableAgent(
    ...
)

agent_a.initiate_chat(agent_b, ...)

# agent_a > agent_b > agent_a with first auto reply > agent_b > agent_a with second auto reply > agent_b > agent_a ends before replying
```

### 4. Human replies with ‘exit’

Please give feedback to agent_a. Press enter to skip and use auto-reply, or type **'exit'** to stop the conversation: **exit**

### 5. GroupChat, no next agent

If the next agent in a GroupChat can’t be determined the chat will end.<br>
If you are customizing the speaker selection method with a Callable, return None to end the chat.

### 6. Swarm, transitioning to end the chat

In a swarm, if you transition to AfterWorkOption.TERMINATE it will end the swarm. The default swarm-level AfterWork option is AfterWorkOption.TERMINATE and this will apply to any agent in the swarm that doesn’t have an AfterWork hand-off specified.<br>
Additionally, if you transition to AfterWorkOption.REVERT_TO_USER but have not specified a user_agent in initiate_swarm_chat then it will end the swarm.

### 7. Reply functions

```python
agent_a = ConversableAgent(
    ...
)

agent_b = ConversableAgent(
    ...
)

def my_reply_func(
    recipient: ConversableAgent,
    messages: Optional[List[Dict]] = None,
    sender: Optional[Agent] = None,
    config: Optional[Any] = None,
) -> Tuple[bool, Union[str, Dict, None]]:
    return True, None # Indicates termination

# Register the reply function as the agent_a's first reply function
agent_a.register_reply(
    trigger=[Agent, None],
    reply_func=my_reply_func,
    position=0

)

agent_a.initiate_chat(agent_b, ...)

# agent_a > agent_b > agent_a ends with first custom reply function
```